# Country preprocessing
Goal: 대륙 | 나라 | 도시 피쳐 생성 (기준: 해당 요청 고객?의 회사 지역?)

---
### 나라 피쳐
1. response_corporate 나라 매핑
   - LGEKR -> 'Korea'
#### 방안 1.
2-1. customer_country의 뒤 / 까지를 나라로 하자.

2-2. cusomer_country의 nan값을 response_corporate의 값으로 채우자.
#### 방안 2.
2-3. response_corporate 그 자체를 사용

### 1. 나라 피쳐

In [3]:
%cd preprocessing_gy

/Users/goodyoung/Desktop/GIt/LG_Aimers/phase2_module/preprocessing_gy


/Users/goodyoung/Desktop/GIt/LG_Aimers/.env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [13]:
import pandas as pd
import pickle
import re
def set_up(mode= None):
    pd.set_option('display.max_rows', mode)

In [14]:
df = pd.read_csv("train.csv")

In [15]:
gy = ['customer_country', 'response_corporate', 'lead_owner', 'customer_idx']

In [16]:
df  = df[gy]

In [17]:
with open('response_corporate.pkl','rb') as f:
    res = pickle.load(f)

In [18]:
df['response_corporate2'] = df['response_corporate'].map(res)

In [19]:
df['customer_country'] = df['customer_country'].str.lower().str.strip()
df['response_corporate2'] = df['response_corporate2'].str.lower().str.strip()

### 2-1.

In [20]:
# 정규표현식으로 나라만 거르기
df.loc[:,"country"] = df["customer_country"].str.extract(r'/([^/]+)$')[0]
df['country'] = df['country'].str.strip()

In [21]:
#숫자 포함 -> none으로 변경 및 결측치 채우기
df['country'] = df['country'].apply(lambda x: 'none' if re.search(r'\d', str(x)) else x)
df['country'].fillna('none',axis = 0, inplace = True)

In [22]:
# 비교 할 때 사용한 코드
# df['test2'] = df['customer_country'].str.split('/').str[-1]
# df['test2'] = df['test2'].apply(lambda x: 'none' if re.search(r'\d', str(x)) else x)
# v = df['test1'].value_counts()
# n = df['test2'].value_counts()
# print(len(v),len(n))
# 두개의 검증 코드
# v_, n_ = list(v.index), list(n.index)
# for i in n_:
#     if i not in v_:
#         print('n_',i)
#     else:
#         v_.remove(i)
# for j in v_:
#     print('v_',j)
# 지금까지로는 정규표현식이 더 많이 압축이 되고, 압축된 데이터도 더 일반화적이라서 정규 표현식을 사용하도록 함

In [23]:
area = {'anguilla':'united kingdom',
'br':'none',
'a':'none',
'nd':'none',
'ny':'none',
'rj':'none',
'us':'united states',
'ca':'none',
'kerela':'india',
'pune':'india',
'colombia - cartagena':'colombia',
'country':'none'}
# none 값

In [24]:
for i,j in area.items():
    idx = df[df['country']==i].index
    if len(idx):
        df.loc[idx,'country'] = j

### 2-2.

In [26]:
# none 결측치 채우기
cond = df['country'] =='none'
df.loc[cond,'country'] = df[cond]['response_corporate2']

In [27]:
df['country'].value_counts()

country
india                 16931
brazil                 9315
united states          5918
mexico                 2759
philippines            2654
                      ...  
swaziland                 1
lucknow                   1
gurgaon                   1
ohio united states        1
chennai                   1
Name: count, Length: 195, dtype: int64

---
### 도시 피쳐
#### 방안 1.
3-1. customer_country의 맨 뒤에서 / 앞자리를 도시로 하자.
#### 방안 2.
3-2. 나라 피쳐로 나온 것 에서 customer_country에서 그 값을 제외 시키면 도시가 나올 것 이다.

### 3-1.

In [28]:
df['city'] = df['customer_country'].str.split('/').str[-2]

In [29]:
df['city'].value_counts()

city
                       15155
são paulo               1553
mumbai                   709
lima                     625
hyderabad                581
                       ...  
thrithallore               1
khanna                     1
hagonoy                    1
antique philippines        1
pisco                      1
Name: count, Length: 10260, dtype: int64

In [30]:
cond = df['city'] ==''
df.loc[cond,'city'] = 'others'

In [31]:
df['city'].fillna('others',axis = 0, inplace = True)

In [32]:
df['city'].value_counts()

city
others                 17340
são paulo               1553
mumbai                   709
lima                     625
hyderabad                581
                       ...  
thrithallore               1
khanna                     1
hagonoy                    1
antique philippines        1
pisco                      1
Name: count, Length: 10259, dtype: int64

---
### 대륙 피쳐
4-1. 나라 피쳐로부터 대륙 매핑.

    - train은 최대한 채우고 test데이터에서 없는 값들은 other?로 처리?

In [33]:
df['country']

0        philippines
1        philippines
2              india
3              india
4              india
            ...     
59294         poland
59295       colombia
59296           peru
59297           peru
59298         france
Name: country, Length: 59299, dtype: object

In [34]:
with open('country_continent.pkl','rb') as f:
    cont = pickle.load(f)

In [35]:
for i,j in enumerate(df['country']):
    if j in cont.keys():
        df.loc[i,'continent'] = cont[j]
    else:
        df.loc[i,'continent'] = 'other'

In [36]:
df['continent'].value_counts()

continent
asia & pacific          21526
latin america           14626
north america            8965
europe                   6019
middle east & africa     4564
other                    3569
russia & cis               30
Name: count, dtype: int64

In [37]:
df

,customer_country,response_corporate,lead_owner,customer_idx,response_corporate2,country,city,continent
0,/quezon city/philippines,LGEPH,0,32160,philippines,philippines,quezon city,asia & pacific
1,/ph-00/philippines,LGEPH,1,23122,philippines,philippines,ph-00,asia & pacific
2,/kolkata /india,LGEIL,2,1755,india,india,kolkata,asia & pacific
3,/bhubaneswar/india,LGEIL,3,4919,india,india,bhubaneswar,asia & pacific
4,/hyderabad/india,LGEIL,4,17126,india,india,hyderabad,asia & pacific
...,...,...,...,...,...,...,...,...
59294,/sląskie/poland,LGEPL,694,33747,poland,poland,sląskie,europe
59295,/bogotá dc /colombia,LGECB,39,35420,colombia,colombia,bogotá dc,latin america
59296,/pisco/peru,LGEPR,125,19249,peru,peru,pisco,latin america
59297,/santa cruz bolivia/peru,LGEPR,134,40327,peru,peru,santa cruz bolivia,latin america


In [40]:
df = pd.read_csv("train.csv")

In [43]:
df[df.duplicated()]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
320,1.00,/East London/South Africa,AS,0.040816,30958,End-Customer,SMB,NaN,NaN,NaN,...,LGESA,more than a year,1,0,0.003079,0.026846,corporate / office,IT/Software,26,False
322,1.00,/East London/South Africa,AS,0.040816,30958,End Customer,SMB,NaN,NaN,NaN,...,LGESA,more than a year,1,0,0.003079,0.026846,corporate / office,IT/Software,26,False
344,1.00,/Medellin /Colombia,AS,0.003937,39065,NaN,SMB,0.0,NaN,NaN,...,LGECB,9 months ~ 1 year,0,0,0.003079,0.026846,corporate / office,NaN,41,False
1253,1.00,/Brisbane/Australia,ID,0.084337,14526,NaN,Enterprise,NaN,1.0,NaN,...,LGEAP,less than 3 months,0,1,0.003079,0.064566,corporate / office,NaN,306,False
1314,1.00,//Ghana,ID,0.075000,4705,Specifier / Influencer,SMB,NaN,1.0,NaN,...,LGEAF,less than 3 months,0,1,0.003079,0.064566,corporate / office,Telecommunication,170,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59134,0.75,//Mexico,ID,0.042254,33676,NaN,SMB,NaN,NaN,NaN,...,LGEMS,less than 3 months,0,0,0.000026,0.031579,public facility,NaN,214,True
59262,1.00,/rio de janeiro /Brazil,AS,0.040000,13618,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGESP,more than a year,0,0,0.000026,0.028777,public facility,Museum / Gallery,97,False
59275,1.00,/Temuco/Chile,AS,0.040000,22100,Specifier/ Influencer,Enterprise,0.0,NaN,NaN,...,LGECL,less than 3 months,0,0,0.000026,0.028777,public facility,Others,100,False
59290,0.75,/Dolnośląskie/Poland,AS,NaN,6739,Specifier/ Influencer,SMB,NaN,NaN,NaN,...,LGEPL,less than 3 months,0,0,0.000026,0.028777,public facility,Others,694,False


In [49]:
df = pd.read_csv("submission.csv")